In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load CSV files
gpt35 = pd.read_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Valeria/23/23_stance_prompts_few_shots_with_answers_GPT3.5.csv')

gpt4 = pd.read_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Valeria/23/23_stance_prompts_few_shots_with_answers_GPT4.csv')


In [ ]:
TOPICS = [
    "abortion",
    "atheism",
    "climate",
    "feminist",
    "hillary",
]

In [ ]:
gpt35['labels'].value_counts()


1    715
2    304
0    230
Name: labels, dtype: int64

In [ ]:
gpt4['labels'].value_counts()

1    715
2    304
0    230
Name: labels, dtype: int64

In [ ]:
def evaluation(df_stats: pd.DataFrame, y_col: str ):
    try:
        results = classification_report(df_stats["labels"], df_stats[y_col].astype(int), output_dict=True)

        f1_against = results['1']['f1-score']
        f1_favor = results['2']['f1-score']
        tweeteval_result = (f1_against+f1_favor) / 2

    except Exception as ex:
        print(f"Issues with task: {ex}")

    return tweeteval_result, results

In [ ]:
gpt4_filtered=gpt4[gpt4['answerOLD']!='none']
columns_to_evaluate = ['answerPrompt2', 'answerPrompt4', 'answerPrompt6', 'answerPrompt8', 'answerPrompt10', 'answerOLD']

dfs = []

for column in columns_to_evaluate:
    _, results = evaluation(gpt4_filtered, column)

    results_df = pd.DataFrame([results])

    results_df['evaluation_column'] = column

    dfs.append(results_df)



all_results_df = pd.concat(dfs, axis=0)
all_results_df.reset_index(inplace=True, drop=True)

print(all_results_df)
all_results_df.to_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Valeria/23/23_stance_resultEvaluation_FILTERED.csv', index=False)

                                                   0  \
0  {'precision': 0.543859649122807, 'recall': 0.8...   
1  {'precision': 0.5138121546961326, 'recall': 0....   
2  {'precision': 0.5258855585831063, 'recall': 0....   
3  {'precision': 0.5356125356125356, 'recall': 0....   
4  {'precision': 0.5362318840579711, 'recall': 0....   
5  {'precision': 0.5492957746478874, 'recall': 0....   

                                                   1  \
0  {'precision': 0.9037433155080213, 'recall': 0....   
1  {'precision': 0.9123134328358209, 'recall': 0....   
2  {'precision': 0.9305019305019305, 'recall': 0....   
3  {'precision': 0.9299242424242424, 'recall': 0....   
4  {'precision': 0.9191729323308271, 'recall': 0....   
5  {'precision': 0.6907801418439716, 'recall': 0....   

                                                   2  accuracy  \
0  {'precision': 0.7286135693215339, 'recall': 0....  0.756844   
1  {'precision': 0.7093023255813954, 'recall': 0....  0.739936   
2  {'precision':

In [ ]:
results_df = pd.DataFrame(columns=['Run', 'F1_Macro_gpt4', 'Accuracy_gpt4'])



gpt4_filtered=gpt4[gpt4['answerOLD']!='none']


y_true = gpt4_filtered['labels'].astype(int)

for run_config in ['answerPrompt2', 'answerPrompt4', 'answerPrompt6', 'answerPrompt8', 'answerPrompt10', 'answerOLD']:


    y_pred = gpt4_filtered[run_config].astype(int)

    conf_matrix = confusion_matrix(y_true, y_pred)

    accuracyGPT4 = accuracy_score(y_true, y_pred)

    f1GPT4 = f1_score(y_true, y_pred, average='macro')

    results_df = results_df.append({
        'Run': run_config,
        'F1_Macro_gpt4': f1GPT4,
        'Accuracy_gpt4': accuracyGPT4
    }, ignore_index=True)

print(results_df)

results_df.to_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Valeria/23/23_stance_resultEvaluation_ALL_STANCE_FILTERED.csv', index=False)

              Run  F1_Macro_gpt4  Accuracy_gpt4
0   answerPrompt2       0.740092       0.756844
1   answerPrompt4       0.723281       0.739936
2   answerPrompt6       0.731529       0.745572
3   answerPrompt8       0.742228       0.756844
4  answerPrompt10       0.735322       0.750403
5       answerOLD       0.562033       0.603865


<ipython-input-66-8df1954e0e85>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-66-8df1954e0e85>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-66-8df1954e0e85>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-66-8df1954e0e85>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-66-8df1954e0e85>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = 

In [ ]:
results_df = pd.DataFrame(columns=['Run', 'F1_Macro_gpt35', 'Accuracy_gpt35', 'F1_Macro_gpt4', 'Accuracy_gpt4'])

y_true = gpt4['labels'].astype(int)


for run_config in ['answerPrompt2', 'answerPrompt4', 'answerPrompt6', 'answerPrompt8', 'answerPrompt10']:


    if run_config!='answerOLD':
      y_pred = gpt35[run_config].astype(int)  # Predictions for the current run configuration


     # Check for NaN values in y_true and y_pred
      nan_positions_true = y_true.isna().any()
      nan_positions_pred = y_pred.isna().any()

      if nan_positions_true or nan_positions_pred:
        print(f"NaN values found in sheet run configuration '{run_config}':")
        print("NaN positions in y_true:")
        print(y_true[y_true.isna()])
        print("\nNaN positions in y_pred:")
        print(y_pred[y_pred.isna()])

      conf_matrix = confusion_matrix(y_true, y_pred)

      accuracyGPT35 = accuracy_score(y_true, y_pred)

      f1GPT35 = f1_score(y_true, y_pred, average='macro')
    else:
      accuracyGPT35 = 0
      f1GPT35 = 0

    y_pred = gpt4[run_config].astype(int)

    conf_matrix = confusion_matrix(y_true, y_pred)

    accuracyGPT4 = accuracy_score(y_true, y_pred)

    f1GPT4 = f1_score(y_true, y_pred, average='macro')

    results_df = results_df.append({
        'Run': run_config,
        'F1_Macro_gpt35': f1GPT35,
        'Accuracy_gpt35': accuracyGPT35,
        'F1_Macro_gpt4': f1GPT4,
        'Accuracy_gpt4': accuracyGPT4
    }, ignore_index=True)

print(results_df)

results_df.to_csv('/content/drive/My Drive/Lavoro ChatGPT/dataset/Valeria/23/23_stance_resultEvaluation_ALL.csv', index=False)

              Run  F1_Macro_gpt35  Accuracy_gpt35  F1_Macro_gpt4  \
0   answerPrompt2        0.584741        0.608487       0.740901   
1   answerPrompt4        0.552045        0.581265       0.724197   
2   answerPrompt6        0.568755        0.599680       0.732306   
3   answerPrompt8        0.563907        0.615693       0.743032   
4  answerPrompt10        0.564652        0.605284       0.736168   

   Accuracy_gpt4  
0       0.756605  
1       0.739792  
2       0.745396  
3       0.756605  
4       0.750200  


<ipython-input-68-f647835a30d7>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-68-f647835a30d7>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-68-f647835a30d7>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-68-f647835a30d7>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
<ipython-input-68-f647835a30d7>:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = 